In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
warnings.simplefilter('ignore')

In [2]:
df = pd.read_csv('cleaned_train.csv')
df.head()

Survived  Pclass  Sex       Age  SibSp  Parch      Fare  Embarked
0         0       3    1 -0.567258      1      0 -0.877591         2
1         1       1    0  0.683970      1      0  1.366380         0
2         1       3    0 -0.254451      0      0 -0.796281         2
3         1       1    0  0.449365      1      0  1.066796         2
4         0       3    1  0.449365      0      0 -0.781901         2

In [3]:
X = df.drop(['Survived'],axis = 1)
y = df['Survived']

In [4]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state= True)

In [6]:
from sklearn.decomposition import PCA
pca_model = PCA(n_components = 0.95)

X_train_pca = pca_model.fit_transform(X_train)
X_test_pca = pca_model.transform(X_test)

In [7]:
pca_model.explained_variance_ratio_

array([0.29655804, 0.2379865 , 0.14446313, 0.11542303, 0.0960734 ,
       0.08225595])

In [8]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV

**Logistic Regression model**

In [9]:

from sklearn.linear_model import LogisticRegression 
lr= LogisticRegression()
lr.fit(X_train_pca,y_train)


# prediction 
ypred_train_lr = lr.predict(X_train_pca)
ypred_test_lr = lr.predict(X_test_pca)
# Evaluation 
print('Train_accuracy:',accuracy_score(y_train,ypred_train_lr))
print('Test Accuracy:',accuracy_score(y_test,ypred_test_lr))
print('Cross validation Score:',cross_val_score(lr,X_train_pca,y_train,cv = 5).mean())

Train_accuracy: 0.7848101265822784
Test Accuracy: 0.8258426966292135
Cross validation Score: 0.781985620013789


In [10]:
# confusion matrix 
print(confusion_matrix(y_test,ypred_test_lr))

[[88 17]
 [14 59]]


In [11]:
# classification Report 
print(classification_report(y_test,ypred_test_lr))

              precision    recall  f1-score   support

           0       0.86      0.84      0.85       105
           1       0.78      0.81      0.79        73

    accuracy                           0.83       178
   macro avg       0.82      0.82      0.82       178
weighted avg       0.83      0.83      0.83       178



In [12]:
from sklearn.metrics import roc_curve ,auc,RocCurveDisplay 
fpr ,tpr,thresholds = roc_curve(y_test,ypred_test_lr)

roc_auc = auc(fpr,tpr)
RocCurveDisplay(fpr =fpr,tpr = tpr,roc_auc = roc_auc).plot()
plt.show()

In [13]:
# AUC Score
from sklearn.metrics import roc_auc_score
print('AUC Score',roc_auc_score(y_test,ypred_test_lr))

AUC Score 0.8231572080887151


**KNN**

In [14]:
from sklearn.neighbors import KNeighborsClassifier

# hyperparameter tunning 
estimator = KNeighborsClassifier()
param_grid = {'n_neighbors':list(range(1,100)),'p':[1,2]}
knn_grid = GridSearchCV(estimator,param_grid,scoring = 'accuracy',cv = 5)
knn_grid.fit(X_train_pca,y_train)

# Best parameter
knn_grid.best_params_

{'n_neighbors': 12, 'p': 1}

In [15]:
# Modelling 
knn_model = knn_grid.best_estimator_
knn_model.fit(X_train_pca,y_train)

# prediction and evaluation on train data 
ypred_train = knn_model.predict(X_train_pca)
ypred_test = knn_model.predict(X_test_pca)

print('Train Accuracy:',accuracy_score(y_train,ypred_train))
print('Test Accuracy:',accuracy_score(y_test,ypred_test))
print('Cross validation Score:',cross_val_score(knn_model,X_train_pca,y_train,cv=5,scoring = 'accuracy').mean())

Train Accuracy: 0.8354430379746836
Test Accuracy: 0.848314606741573
Cross validation Score: 0.8199448438885059


**SVM**

In [16]:
from sklearn.svm import SVC

In [17]:
estimator = SVC()
# param grid
param_grid = {'C':[1,2,3,4,5,8,9,10],'kernel':['linear','poly','rbf','sigmoid']}
svm_grid = GridSearchCV(estimator,param_grid,cv = 5,scoring = 'accuracy')
svm_grid.fit(X_train_pca,y_train)

# best parameter 
print(svm_grid.best_params_)
svm_grid.best_estimator_

{'C': 2, 'kernel': 'rbf'}


SVC(C=2)

In [18]:
# Modelling 
svm_model = svm_grid.best_estimator_
svm_model.fit(X_train_pca,y_train)

# prediction and evaluation on train data

ypred_train = svm_model.predict(X_train_pca)
ypred_test = svm_model.predict(X_test_pca)

print('Train Accuracy:',accuracy_score(y_train,ypred_train))
print('Test Accuracy:',accuracy_score(y_test,ypred_test))
print('Cross validation Score:',cross_val_score(svm_model,X_train_pca,y_train,scoring = 'accuracy',cv = 5).mean())

Train Accuracy: 0.8382559774964838
Test Accuracy: 0.8426966292134831
Cross validation Score: 0.8143011917659806


**Decision Tree**

In [19]:
from sklearn.tree import DecisionTreeClassifier

In [20]:
model = DecisionTreeClassifier(random_state = True)
model.fit(X_train_pca,y_train)

DecisionTreeClassifier(random_state=True)

In [23]:
feature_names = [f'PC{i+1}' for i in range(X_train_pca.shape[1])]


In [24]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

plt.figure(figsize=(30, 30), dpi=600)
plot_tree(model, 
          filled=True,
          feature_names=feature_names,
          class_names=['0', '1'])
plt.savefig(r'C:\Users\sahur\OneDrive\Desktop\Decision Treee\titenic_DT_pca.png')
plt.show()


In [25]:
# Hyperparameter Tunning 
estimator = DecisionTreeClassifier(random_state = True)
param_grid = {'criterion':['gini','entropy'],
             'max_depth':list(range(1,16))}
dt_grid = GridSearchCV(estimator,param_grid,cv = 5 ,scoring = 'accuracy')
dt_grid.fit(X_train_pca ,y_train)

# best parameter 
print(dt_grid.best_params_)
dt = dt_grid.best_estimator_
dt

{'criterion': 'gini', 'max_depth': 4}


DecisionTreeClassifier(max_depth=4, random_state=True)

In [26]:
# important feature 
feats_ab =pd.DataFrame(data = dt.feature_importances_,
                      index = feature_names,
                      columns = ['Important'])
important_feature_dt = feats_ab[feats_ab['Important']>0].index.tolist()
important_feature_dt

['PC1', 'PC2', 'PC3', 'PC4', 'PC5']

**Decision Tree with important columns**

In [27]:
# selection train and test data 
#  Name the PCA components (features)
feature_names = [f'PC{i+1}' for i in range(X_train_pca.shape[1])]

#  Create DataFrame
X_train_pca_df = pd.DataFrame(X_train_pca, columns=feature_names)
X_test_pca_df = pd.DataFrame(X_test_pca, columns=feature_names)

# Now use column names
X_train_dt = X_train_pca_df[important_feature_dt]
X_test_dt = X_test_pca_df[important_feature_dt]

# modelling 
dt = dt_grid.best_estimator_
dt.fit(X_train_dt,y_train)

# prediction and evaluation 
ypred_train = dt.predict(X_train_dt)
ypred_test = dt.predict(X_test_dt)

print('Trian Accuracy :',accuracy_score(y_train,ypred_train))
print('Test Accuracy :',accuracy_score(y_test,ypred_test))
print('Cross validation Score:',cross_val_score(dt,X_train_dt,y_train,cv=5,scoring = 'accuracy').mean())


Trian Accuracy : 0.8326300984528833
Test Accuracy : 0.7752808988764045
Cross validation Score: 0.7693785088151286



**Random Forest Classifier**

In [28]:
from sklearn.ensemble import RandomForestClassifier


In [29]:
estimator = RandomForestClassifier(random_state = True)
param_grid = {'n_estimators':list(range(1,51))}
rf_grid = GridSearchCV(estimator,param_grid,scoring = 'accuracy',cv=5)
rf_grid.fit(X_train_pca,y_train)
rf = rf_grid.best_estimator_
rf

RandomForestClassifier(n_estimators=37, random_state=True)

In [ ]:
feature_names = [f'PC{i+1}' for i in range(X_train_pca.shape[1])]


In [33]:
# Important features 
feats_ab = pd.DataFrame(data = rf.feature_importances_,
                       index = feature_names,
                       columns = ['Importance'])
important_features_ab = feats_ab[feats_ab['Importance']>0].index.tolist()
important_features_ab

['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6']

In [43]:
#  Name the PCA components (features)
feature_names = [f'PC{i+1}' for i in range(X_train_pca.shape[1])]

#  Create DataFrame
X_train_pca_df = pd.DataFrame(X_train_pca, columns=feature_names)
X_test_pca_df = pd.DataFrame(X_test_pca, columns=feature_names)

# Now use column names
X_train_rf = X_train_pca_df[important_feature_dt]
X_test_rf= X_test_pca_df[important_feature_dt]

# Modelling 
rf = rf_grid.best_estimator_
rf.fit(X_train_ab,y_train)

# Evaluation 
ypred_train = rf.predict(X_train_ab)
ypred_test = rf.predict(X_test_ab)

print('Train Accuracy:',accuracy_score(y_train,ypred_train))
print('Test Accuracy:',accuracy_score(y_test,ypred_test))
print('Cross validation Score:',cross_val_score(rf,X_train_rf,y_train,cv=5).mean())

Train Accuracy: 0.9845288326300985
Test Accuracy: 0.7640449438202247
Cross validation Score: 0.7777701172067369


**AdaBoost Classifier**

In [44]:
from sklearn.ensemble import AdaBoostClassifier

In [45]:
estimator = AdaBoostClassifier(random_state = True)
param_grid = {'n_estimators':list(range(1,51))}
ab_grid = GridSearchCV(estimator,param_grid,scoring = 'accuracy',cv=5)
ab_grid.fit(X_train_pca,y_train)

ab = ab_grid.best_estimator_
ab

AdaBoostClassifier(n_estimators=18, random_state=True)

In [47]:
# Important features
feats_ab = pd.DataFrame(data = ab.feature_importances_,
                       index = feature_names,
                       columns = ['Importance'])
important_features_ab = feats_ab[feats_ab['Importance']>0].index.tolist()
important_features_ab

['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6']

In [52]:
#  Name the PCA components (features)
feature_names = [f'PC{i+1}' for i in range(X_train_pca.shape[1])]

#  Create DataFrame
X_train_pca_df = pd.DataFrame(X_train_pca, columns=feature_names)
X_test_pca_df = pd.DataFrame(X_test_pca, columns=feature_names)

# Now use column names
X_train_rf = X_train_pca_df[important_feature_dt]
X_test_rf= X_test_pca_df[important_feature_dt]

# Modelling 
ab = ab_grid.best_estimator_
ab.fit(X_train_ab,y_train)

# Evaluation 
ypred_train = ab.predict(X_train_ab)
ypred_test = ab.predict(X_test_ab)

print('Train Accuracy:',accuracy_score(y_train,ypred_train))
print('Test Accuracy:',accuracy_score(y_test,ypred_test))
print('Cross validation Score:',cross_val_score(ab,X_train_rf,y_train,cv=5).mean())

Train Accuracy: 0.8157524613220816
Test Accuracy: 0.7921348314606742
Cross validation Score: 0.7749532157982862


**Gradient Boost Classifier**

In [54]:
from sklearn.ensemble import GradientBoostingClassifier

In [55]:
estimator = GradientBoostingClassifier(random_state = True)
param_grid = {'n_estimators':list(range(1,10)),
             'learning_rate':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]}
gb_grid = GridSearchCV(estimator,param_grid,scoring = 'accuracy',cv = 5)
gb_grid.fit(X_train_pca,y_train)

gb = gb_grid.best_estimator_
gb

GradientBoostingClassifier(learning_rate=0.3, n_estimators=9, random_state=True)

In [57]:
# important feature
feats_gb = pd.DataFrame(data = gb.feature_importances_,
                       index = feature_names,
                       columns = ['Importance'])

important_feature_gb = feats_gb[feats_gb['Importance']>0].index.tolist()
important_feature_gb

['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6']

In [73]:
#  Name the PCA components (features)
feature_names = [f'PC{i+1}' for i in range(X_train_pca.shape[1])]

#  Create DataFrame
X_train_pca_df = pd.DataFrame(X_train_pca, columns=feature_names)
X_test_pca_df = pd.DataFrame(X_test_pca, columns=feature_names)

# Now use column names
X_train_gb = X_train_pca_df[important_feature_dt]
X_test_gb= X_test_pca_df[important_feature_dt]

# Modelling 
gb = gb_grid.best_estimator_
gb.fit(X_train_gb,y_train)

# prediction 
ypred_train = gb.predict(X_train_gb)
ypred_test = gb.predict(X_test_gb)

# Evaluation 

print('Train Accuracy:',accuracy_score(y_train,ypred_train))
print('Test Accuracy:',accuracy_score(y_test,ypred_test))
print('Cross validation Score:',cross_val_score(gb,X_train_gb,y_train,cv = 5).mean())

Train Accuracy: 0.8762306610407876
Test Accuracy: 0.7808988764044944
Cross validation Score: 0.8045109819757708


**XGBoost Classifier**

In [75]:
# Hyperparameter tunning 
from xgboost import XGBClassifier
estimator = XGBClassifier()
param_grid = {'n_estimators':[10,20,30,40,100],
             'max_depth':[3,4,5],
             'gamma':[0,0.15,0.3,0.5,1]}
xgb_grid = GridSearchCV(estimator,param_grid,scoring = 'accuracy',cv = 5)
xgb_grid.fit(X_train_pca,y_train)

xgb = xgb_grid.best_estimator_
xgb

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=0, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=10,
              n_jobs=None, num_parallel_tree=None, ...)

In [76]:
# Important Features
feats_xgb = pd.DataFrame(data = xgb.feature_importances_,
                        index = feature_names,
                        columns = ['Importance'])
important_features_xgb = feats_xgb[feats_xgb['Importance']>0].index.tolist()
important_features_xgb

['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6']

In [77]:
#  Name the PCA components (features)
feature_names = [f'PC{i+1}' for i in range(X_train_pca.shape[1])]

#  Create DataFrame
X_train_pca_df = pd.DataFrame(X_train_pca, columns=feature_names)
X_test_pca_df = pd.DataFrame(X_test_pca, columns=feature_names)

# Now use column names
X_train_xgb = X_train_pca_df[important_feature_dt]
X_test_xgb= X_test_pca_df[important_feature_dt]

# Modelling 
xgb = xgb_grid.best_estimator_
xgb.fit(X_train_xgb,y_train)

# Evaluation 
ypred_train = xgb.predict(X_train_xgb)
ypred_test = xgb.predict(X_test_xgb)

print('Train Accuracy:',accuracy_score(y_train,ypred_train))
print('Test Accuracy:',accuracy_score(y_test,ypred_test))
print('Cross validation Score:',cross_val_score(xgb,X_train_xgb,y_train,cv = 5).mean())

Train Accuracy: 0.8860759493670886
Test Accuracy: 0.7865168539325843
Cross validation Score: 0.7931842805082241
